In [ ]:
#!pip install import_ipynb

In [1]:
import sys

from PyQt5.QtWidgets import *
from PyQt5.QtCore import *
from PyQt5 import uic

In [2]:
import import_ipynb
import everytime_crawling as ec

importing Jupyter notebook from everytime_crawling.ipynb


In [3]:
board_dic = ec.load_board_names('게시판명.txt')
board_ls = list(board_dic.keys())    
base_url = 'https://everytime.kr'
login_url = 'https://everytime.kr/login'
driver = None

class CrawlingControl(QDialog): #crawling control하는 dialog, 해당 게시판과 그 게시판에 대한 질의에 대해 게시물 크롤링
    def __init__(self, driver, bname):
        super().__init__()
        
        self.setWindowTitle('Crawling Window')
        self.setGeometry(100, 100, 200, 100)
        self.driver = driver
        
        self.layout = QVBoxLayout()
        self.subLayout = QHBoxLayout()
        
        self.query_edit = QLineEdit()
        font = self.query_edit.font()
        font.setPointSize(20)
        self.query_edit.setFont(font)
        self.query_edit.setPlaceholderText("검색어")
        
        self.prog_label = QLabel(bname, self)
        self.btnCancel = QPushButton("취소")
        self.btnAddQuery = QPushButton("검색어 추가")
        
        self.layout.addWidget(self.prog_label)
        self.layout.addWidget(self.query_edit)
        
        self.subLayout.addWidget(self.btnAddQuery)
        self.subLayout.addWidget(self.btnCancel)
        
        self.layout.addLayout(self.subLayout)
        
        self.btnCancel.clicked.connect(self.onCancelButtonClicked)
        self.btnAddQuery.clicked.connect(self.onAddQueryButtonClicked)
#         cns_data = ec.counsel_data(self.driver)
#         cns_data.start_crawling()
        self.setLayout(self.layout)
    def onCancelButtonClicked(self):
        self.reject()
    def onAddQueryButtonClicked(self):
        global base_url
        base_url += '/all/'+self.query_edit.text()
        print(base_url)

def add_url(text):
    global base_url
    base_url += '/'+text

def reset_url():
    global base_url
    base_url = 'https://everytime.kr'


class Item(QWidget):
    def __init__(self, text):
        
        QWidget.__init__(self, flags=Qt.Widget)
        layout = QBoxLayout(QBoxLayout.TopToBottom)
        pb = QPushButton(text)
        layout.addWidget(pb)
        layout.setSizeConstraint(QBoxLayout.SetFixedSize)
        self.setLayout(layout)
        pb.clicked.connect(self.bname_clicked)
        self.text = text
        
    def bname_clicked(self):
        global board_dic
        print(self.text)
        reset_url()
        add_url(board_dic[self.text])
        print(base_url)
        F = CrawlingControl(driver, self.text)
        F.exec_()

class MainWidget(QWidget): 
    def __init__(self): 
        super().__init__() 
        
        #widget setting
        self.layout = QVBoxLayout(self)
        self.layout.addStretch(1)
        
        self.tabs = QTabWidget()
        self.tab1 = QWidget()
        self.tab2 = QWidget()
        self.tab3 = QWidget()
        self.tabs.resize(500,400)
        self.tab1_def()
        self.tab2_def()
        self.tab3_def()
        self.tabs.addTab(self.tab1, "메인창")
        self.tabs.addTab(self.tab2, "게시판설정")
        self.tabs.addTab(self.tab3, "크롤링 메뉴")
        
        self.layout.addWidget(self.tabs)
        self.setLayout(self.layout)
        
        
    def tab1_def(self):    
        self.tab1.layout = QVBoxLayout(self)
        id_edit = QLineEdit()
        font = id_edit.font()
        font.setPointSize(20)
        id_edit.setFont(font)
        id_edit.setPlaceholderText("아이디")
        self.tab1.id_edit = id_edit
        
        pw_edit = QLineEdit()
        font = pw_edit.font()
        font.setPointSize(20)
        pw_edit.setFont(font)
        pw_edit.setPlaceholderText("비밀번호")
        pw_edit.setEchoMode(QLineEdit.Password) #비밀번호 가리기
        self.tab1.pw_edit = pw_edit
        
        btn_login = QPushButton('에타 로그인') 
        btn_close = QPushButton('종료') 
        
        #widget 추가
        self.tab1.layout.addWidget(id_edit)
        self.tab1.layout.addWidget(pw_edit)
        
        
        subLayout = QHBoxLayout() 
        subLayout.addWidget(btn_login) 
        subLayout.addWidget(btn_close) 
        
        self.tab1.setLayout(self.tab1.layout) 
        self.tab1.setGeometry(300, 100, 350, 150) # x, y, width, height 
        #self.tab1.setWindowTitle("QWidget") 
        
        self.tab1.layout.addLayout(subLayout)
        
        #self.show()
        
        btn_login.clicked.connect(self.btn_login_clicked)
        btn_close.clicked.connect(self.close_btn_clicked)
    
    def tab2_def(self):
        self.tab2.layout = QBoxLayout(QBoxLayout.TopToBottom)
        self.tab2.viewer = QListWidget(self)
        self.tab2.layout.addWidget(self.tab2.viewer)
        self.tab2.setLayout(self.tab2.layout)
        
        global board_ls
        
        for i, bname in enumerate(board_ls):
            self.tab2.item = QListWidgetItem(self.tab2.viewer)
            custom_widget = Item(bname)

            # item은 custom_widget의 사이즈를 알지 못하므로 알려줘야 한다
            self.tab2.item.setSizeHint(custom_widget.sizeHint())
            self.tab2.viewer.setItemWidget(self.tab2.item, custom_widget)
            self.tab2.viewer.addItem(self.tab2.item)
        
    def btn_login_clicked(self):
#         F = Login()
#         F.exec_()
        global driver
        _id = self.tab1.id_edit.text()
        _pw = self.tab1.pw_edit.text()
        driver = ec.login_qt(_id, _pw)
        
        #self.accept()
        
    def close_btn_clicked(self):
        global driver
        try:
            driver.close()
        except:
            pass
        QCoreApplication.instance().quit()
        
    
    def tab3_def(self):
        self.timerVar = QTimer()
        
        
        self.tab3.layout = QVBoxLayout(self)
        self.tab3.subLayout = QHBoxLayout(self)
        
        
        self.tab3.i = 0
        self.tab3.label = "진행 경과\n%d개" % self.tab3.i
        
        self.tab3.prog_label = QLabel(self.tab3.label, self)
        btn_contents = QPushButton('게시글 크롤링')
        btn_comments = QPushButton('댓글 크롤링')
        
        self.tab3.subLayout.addWidget(btn_contents)
        self.tab3.subLayout.addWidget(btn_comments)
        btn_contents.clicked.connect(self.btn_contents_clicked)
        
        
        self.tab3.layout.addWidget(self.tab3.prog_label)
        
        
        self.tab3.layout.addLayout(self.tab3.subLayout)
        self.tab3.setLayout(self.tab3.layout)
        
    

    def btn_contents_clicked(self):
        
        self.timerVar.setInterval(5000)
        self.timerVar.timeout.connect(self.checkByTime)
        self.timerVar.start()
    
    def checkByTime(self) :
        self.tab3.i += 1
        print(self.tab3.i)
        self.tab3.prog_label.setText(self.tab3.label)

        
        
        
   

In [4]:
import sys

from PyQt5.QtWidgets import *
from PyQt5.QtCore import *
from PyQt5 import uic




    
    
    

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.initUI()
        
    def initUI(self):
        self.setWindowTitle('Main Window')
        self.setGeometry(100, 100, 300, 200)

 
        
        
        self.wg = MainWidget() # wg = MyWidget2() 
        # placeholder -- QWidget 상속하여 만든것으로 추후 교체하면 됨. 
        self.setCentralWidget(self.wg) # 반드시 필요함. 
        self.setGeometry(400, 700, 600, 270) 
        self.show()
        
        #centralWidget = QWidget()
        #centralWidget.setLayout(layout)





if __name__ == "__main__":
    app = QApplication(sys.argv)
    mains = MainWindow()
    mains.show()
    #form = Form()
    #form.show()
    exit(app.exec_())

1
2


# 연습

In [4]:
class Login(QDialog):
    def __init__(self):
        super().__init__()
        self.initUI()
 
    def initUI(self):
        self.setWindowTitle('Sub Window')
        self.setGeometry(100, 100, 200, 100)
         
        #widget setting
        layout = QVBoxLayout()
        layout.addStretch(1)
        
        
        id_edit = QLineEdit()
        font = id_edit.font()
        font.setPointSize(20)
        id_edit.setFont(font)
        id_edit.setPlaceholderText("아이디")
        self.id_edit = id_edit
        
        pw_edit = QLineEdit()
        font = pw_edit.font()
        font.setPointSize(20)
        pw_edit.setFont(font)
        pw_edit.setPlaceholderText("비밀번호")
        pw_edit.setEchoMode(QLineEdit.Password) #비밀번호 가리기
        self.pw_edit = pw_edit
 
        subLayout = QHBoxLayout()
        
        btnOK = QPushButton("게시물 크롤링 시작")
        btnOK.clicked.connect(self.onOKButtonClicked)
 
        btnCancel = QPushButton("취소")
        btnCancel.clicked.connect(self.onCancelButtonClicked)
 
        #widget 추가
        layout.addWidget(id_edit)
        layout.addWidget(pw_edit)
        
        subLayout.addWidget(btnOK)
        subLayout.addWidget(btnCancel)
        layout.addLayout(subLayout)
 
        layout.addStretch(1)
 
        self.setLayout(layout)
 
    def onOKButtonClicked(self):
        _id = self.id_edit.text()
        _pw = self.pw_edit.text()
        driver = ec.login_qt(_id, _pw)
        F = CrawlingControl(driver)
        F.exec_()
        #self.accept()
        
 
    def onCancelButtonClicked(self):
        self.reject()
 
    def showModal(self):
        return super().exec_()
    
    
class Item(QWidget):
    def __init__(self, text):
        
        QWidget.__init__(self, flags=Qt.Widget)
        layout = QBoxLayout(QBoxLayout.TopToBottom)
        pb = QPushButton(text)
        layout.addWidget(pb)
        layout.setSizeConstraint(QBoxLayout.SetFixedSize)
        self.setLayout(layout)
        pb.clicked.connect(self.bname_clicked)
        self.text = text
        
    def bname_clicked(self):
        global board_dic
        print(self.text)
        reset_url()
        add_url(board_dic[self.text])
        print(base_url)
        F = Form2()
        F.exec_()
        

class Form(QDialog):
    def __init__(self):
        #QWidget.__init__(self, flags=Qt.Widget)
        QDialog.__init__(self, None)
        self.setWindowTitle("QTreeWidget")
        layout = QBoxLayout(QBoxLayout.TopToBottom)
        self.viewer = QListWidget(self)
        layout.addWidget(self.viewer)
        self.setLayout(layout)
        
        global board_ls
        
        
        for i, bname in enumerate(board_ls):
            item = QListWidgetItem(self.viewer)
            custom_widget = Item(bname)

            # item은 custom_widget의 사이즈를 알지 못하므로 알려줘야 한다
            item.setSizeHint(custom_widget.sizeHint())
            self.viewer.setItemWidget(item, custom_widget)
            self.viewer.addItem(item)
            
class Form2(QDialog):
    def __init__(self):
        QDialog.__init__(self, None)
        self.setWindowTitle("QTreeWidget")
        layout = QBoxLayout(QBoxLayout.TopToBottom)
        self.viewer = QListWidget(self)
        layout.addWidget(self.viewer)
        self.setLayout(layout)
        
        ec.login()
        
        for i, bname in enumerate(board_ls):
            item = QListWidgetItem(self.viewer)
            custom_widget = Item(bname)

            # item은 custom_widget의 사이즈를 알지 못하므로 알려줘야 한다
            item.setSizeHint(custom_widget.sizeHint())
            self.viewer.setItemWidget(item, custom_widget)
            self.viewer.addItem(item)
            